In [8]:
+import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Вариант 2
    Задание 3
## $\frac{\partial u}{\partial t} = \frac{\partial}{\partial x}(u^{1/2}\frac{\partial u}{\partial x}) + \frac{\partial}{\partial y}(u^{1/2}\frac{\partial u}{\partial y}), 0<t\leq 1, 0<x,y<1;$  
 
 $u(0,x,y) = (1+x+y)^4/441, 0 \leq x,y \leq 1;$  
 $u(t,0,y) = (1+y)^4/(21-20t)^2, 0<t\leq 1, 0 < y \leq 1;$   
 $u(t,1,y) = (2+y)^4/(21-20t)^2, 0<t\leq 1, 0 < y \leq 1;$  
 $u(t,x,0) = (1+x)^4/(21-20t)^2, 0<t\leq 1, 0 < x < 1;$  
 $u(t,x,1) = (2+x)^4/(21-20t)^2, 0<t\leq 1, 0 < x \leq 1;$

In [9]:
nu = 1/2
C_z = 1
C_t = 21

## Аналитическое решение

In [10]:
def analyt(t,x,y):
    return (C_z + x + y)**(2/nu)*(C_t - 4*t*(nu+2)/nu)**(-1/nu)

# Численное решение

In [67]:
###
delta = 0.0
eps = 0.0001
N = 1281
L = 81
M = 81
###
xl, hx = np.linspace(0, 1, L, retstep=True)
ym, hy = np.linspace(0, 1, M, retstep=True)
tn, dt = np.linspace(0, 1, N, retstep=True)

def run(U_t, U_t_next):
    U_k = U_t
    u_next = np.empty(shape=(L,M))

    max_diff = eps + 1

    while max_diff > eps:
        u_next = np.empty(shape=(L,M))
        u_next[ : , 0] = U_t_next[: , 0]
        u_next[ : , -1] = U_t_next[: , -1]
        u_next[ 0 , :] = U_t_next[0 , :]
        u_next[ -1 , :] = U_t_next[-1 , :]
        for m in range(1, M-1):
            a = np.array([-dt/(2*hx**2)*((U_k[l+1][m])**nu + (U_k[l][m])**nu) for l in range(1, L-1)])
            c = np.array([-dt/(2*hx**2)*((U_k[l-1][m])**nu + (U_k[l][m])**nu) for l in range(1, L-1)])
            b = -a-c+1
            d = U_t[1:-1, m]
            
            alpha = [-a[0] / b[0]]
            beta = [(d[0] - c[0] * U_t_next[0][m]) / b[0]]
            for l in range(1, L - 2):
                alpha.append(-a[l] / (b[l] + c[l]*alpha[l-1])) #28
                beta.append((d[l] - c[l]*beta[l-1]) / (b[l] + c[l]*alpha[l-1]))
            
            for l in range(L - 2, 0, -1):
                u_next[l][m] = alpha[l - 1] * u_next[l + 1][m] + beta[l - 1]

        for l in range(1, L-1):
            a = np.array([-dt/(2*hy**2)*((U_k[l][m+1])**nu + (U_k[l][m])**nu) for m in range(1, M-1)])
            c = np.array([-dt/(2*hy**2)*((U_k[l][m-1])**nu + (U_k[l][m])**nu) for m in range(1, M-1)])
            b = -a-c+1
            d = u_next[l, 1:-1]

            alpha = [-a[0] / b[0]]
            beta = [(d[0] - c[0] * U_t_next[l][0]) / b[0]]

            for l in range(1, M - 2):
                alpha.append(-a[l] / (b[l] + c[l]*alpha[l-1]))
                beta.append((d[l] - c[l]*beta[l-1]) / (b[l] + c[l]*alpha[l-1]))

            for m in range(M - 2, 0, -1):
                u_next[l][m] = alpha[m - 1] * u_next[l][m + 1] + beta[m - 1]

        max_diff = np.max(np.abs((u_next - U_k)[1:-1] / u_next[1:-1]))
        U_k = u_next
        
    return U_k

In [68]:
U = np.zeros(shape=(N, L, M))
U[0] = (C_z + xl + ym)**(2/nu)*C_t**(-1/nu)
for n in range(0, N):
    U[n, 0, :] = (C_z + ym)**(2/nu)*(C_t - 4*(nu+2)*tn[n]/nu)**(-1/nu)
    U[n, -1, :] = (C_z + 1 + ym)**(2/nu)*(C_t - 4*(nu+2)*tn[n]/nu)**(-1/nu)
    U[n, :, 0] = (C_z + xl)**(2/nu)*(C_t - 4*(nu+2)*tn[n]/nu)**(-1/nu)
    U[n, :, -1] = (C_z + 1 + xl)**(2/nu)*(C_t - 4*(nu+2)*tn[n]/nu)**(-1/nu)


In [69]:
for n, t in enumerate(tn[1:], 1):
        U[n] = run(U[n-1],U[n])
u_a = analyt(tn[-1], xl, ym[1])
u = U[-1, :, 1]


step_for_output = (L - 1) // 5


In [70]:
tf = pd.DataFrame(columns = ym[::step_for_output], index = xl[::step_for_output])

In [71]:
step_for_output

16

In [72]:
ym[::step_for_output]

array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

In [73]:
tf.iloc[:, :] = U[-1, ::step_for_output, ::step_for_output]

In [74]:
tf

,0.0,0.2,0.4,0.6,0.8,1.0
0.0,1.0000,2.073600,3.841600,6.553600,10.497600,16.0000
0.2,2.0736,1.838681,3.658965,6.547292,10.834958,23.4256
0.4,3.8416,3.565100,6.257840,10.269162,15.967181,33.1776
0.6,6.5536,7.313111,11.620615,17.625105,25.726827,45.6976
0.8,10.4976,13.670447,20.316639,29.148992,40.601859,61.4656
1.0,16.0000,23.425600,33.177600,45.697600,61.465600,81.0000


In [75]:
tf_analytc = pd.DataFrame(columns = ym[::step_for_output], index = xl[::step_for_output])
a=0
for i in range(0, L, step_for_output): #L=M
    tf_analytc.iloc[:, a] = analyt(1, xl[::step_for_output], ym[i])
    a = a + 1

In [76]:
tf_analytc

,0.0,0.2,0.4,0.6,0.8,1.0
0.0,1.0000,2.0736,3.8416,6.5536,10.4976,16.0000
0.2,2.0736,3.8416,6.5536,10.4976,16.0000,23.4256
0.4,3.8416,6.5536,10.4976,16.0000,23.4256,33.1776
0.6,6.5536,10.4976,16.0000,23.4256,33.1776,45.6976
0.8,10.4976,16.0000,23.4256,33.1776,45.6976,61.4656
1.0,16.0000,23.4256,33.1776,45.6976,61.4656,81.0000


In [77]:
diff_ = np.max(np.abs(tf.to_numpy() - tf_analytc.to_numpy()))

In [78]:
diff_

7.4584187224221346

In [79]:
dff = tf - tf_analytc

In [80]:
dff

,0.0,0.2,0.4,0.6,0.8,1.0
0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0.2,0.0,-2.002919,-2.894635,-3.950308,-5.165042,0.0
0.4,0.0,-2.988500,-4.239760,-5.730838,-7.458419,0.0
0.6,0.0,-3.184489,-4.379385,-5.800495,-7.450773,0.0
0.8,0.0,-2.329553,-3.108961,-4.028608,-5.095741,0.0
1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
